In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sentencefeaturesoov', 'crawl_emb_nocomp.pickle', 'jigsaw-unintended-bias-in-toxicity-classification', 'crawl_emb_processed_lz4.joblib', 'x-train-tokenized', 'crawl_emb_nocomp.joblib', 'crawl_emb_processed.joblib', 'bert-pretrained-models', 'fasttext-crawl-300d-2m', 'glove840b300dtxt', 'roov-crawl.pickle']


In [4]:
LSTM_UNITS = 128
batch_size = 512
n_seeds = 1
n_splits = 10
n_epochs = 20
EMBED_SIZE = 300
SUBGROUP_NEGATIVE_WEIGHT_COEF = 1
BACKGROUND_POSITIVE_WEIGHT_COEF = 0

MAX_LEN = 220

EMB_DROPOUT = 0.3
MIDDLE_DROPOUT = 0.3

DEBUG = False
if DEBUG:
    n_seeds = 2
    n_splits = 2
    n_epochs = 3
DEBUG_DATA_SIZE = 10000

In [8]:
import pandas as pd
import gc

In [6]:
%%time

import re
# これだと、'はembeddingに結構入ってるのに除外されちゃう。　よくないので ' だけ抜いた
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x: str) -> str:
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' {} '.format(punct))
    return x

# 9.9G


if DEBUG:
    DEBUG_DATA_SIZE = 10000
    train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', nrows=DEBUG_DATA_SIZE)
else:
    train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')

x_train = train['comment_text'].apply(lambda x: clean_text(x))

CPU times: user 20 s, sys: 779 ms, total: 20.8 s
Wall time: 19.5 s


In [ ]:
del train

In [11]:
gc.collect()

317

In [1]:
import textblob
TextBlob = textblob.TextBlob

In [ ]:
en_ja_list = []
for text in tqdm(x_train):
    try:
        en_ja_list.append(str(TextBlob(text).translate(to='ja').translate(to='en')))
    except textblob.exceptions.TextBlobError:
        en_ja_list.append(text)

  0%|          | 380/1804874 [03:50<252:07:19,  1.99it/s]

In [ ]:
with open('tmp.txt', 'w') as f:
    f.write('\n'.join(en_ja_list))